# Validation of the analytical model with numerical data + comparison with Heck et al. (2023)

 This is an interactive version of Figures 8,9,11,12 found in Tamaro et al (in review). 

## Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import fsolve

 Define function to compute $a$

In [ ]:
def find_a(x,*data):
    C_T_primo,gamma = data
    u_4 = 1 - 0.5*C_T_primo*(1-x)*np.cos(np.deg2rad(gamma))**2
    v_4 = -0.25*C_T_primo*(1-x)**2*np.sin(np.deg2rad(gamma))*np.cos(np.deg2rad(gamma))**2
    rhs = 1 - ( np.sqrt(1-u_4**2-v_4**2) )/(np.sqrt(C_T_primo)*np.cos(np.deg2rad(gamma)))
    # c=3/0
    # print(x)
    return x - rhs

 Define function to compute $a$

In [ ]:
def find_a_con_CT(x,*data):
    C_T,gamma = data
    C_T_primo = C_T/(1-x)**2/np.cos(np.deg2rad(gamma))**2
    u_4 = 1 - 0.5*C_T_primo*(1-x)*np.cos(np.deg2rad(gamma))**2
    v_4 = -0.25*C_T_primo*(1-x)**2*np.sin(np.deg2rad(gamma))*np.cos(np.deg2rad(gamma))**2
    rhs = 1 - ( np.sqrt(1-u_4**2-v_4**2) )/(np.sqrt(C_T_primo)*np.cos(np.deg2rad(gamma)))
    # c=3/0
    # print(x)
    return x - rhs

In [ ]:
def Heck(C_T_primo,gamma):
    x0 = C_T_primo/(C_T_primo+4)
    data = (C_T_primo,gamma) 
    a_n = fsolve(find_a, x0,args=data,factor=0.1)[0]
    eta_P = ( (1+0.25*C_T_primo)*(1-a_n)*np.cos(np.deg2rad(gamma)) )**3
    eta_T = ( (1+0.25*C_T_primo)*(1-a_n)*np.cos(np.deg2rad(gamma)) )**2
    return eta_P,eta_T

In [ ]:
def Heck_w_C_T(C_T,gamma):
    x0 = 0.2
    data = (C_T,gamma) 
    a_n = fsolve(find_a_con_CT, x0,args=data,factor=0.1)[0]
    C_T_primo = C_T/((1-a_n)**2*np.cos(np.deg2rad(gamma))**2)
    
    eta_P = ( (1+0.25*C_T_primo)*(1-a_n)*np.cos(np.deg2rad(gamma)) )**3
    eta_T = ( (1+0.25*C_T_primo)*(1-a_n)*np.cos(np.deg2rad(gamma)) )**2

    # print('ecco: ' + str(C_T_primo))
    return eta_P,eta_T,a_n,C_T_primo

## Define function to compute $a$

In [ ]:
def find_a(C_T,mu):
    sinMu = np.sin(mu)
    a = (1- ( (1+np.sqrt(1-C_T-1/16*C_T**2*sinMu**2))/(2*(1+1/16*C_T*sinMu**2))) )    
    return a

 Define function to compute $C_T$

In [ ]:
def find_ct(x,*data):
    sigma,cd,cl_alfa,gamma,delta,k,cosMu,sinMu,tsr,theta,mu = data
    CD = np.cos(np.deg2rad(delta))
    CG = np.cos(np.deg2rad(gamma))
    SD = np.sin(np.deg2rad(delta))
    SG = np.sin(np.deg2rad(gamma))
    a = (1- ( (1+np.sqrt(1-x-1/16*x**2*sinMu**2))/(2*(1+1/16*x*sinMu**2))) )    
    k_1s = -1*(15*np.pi/32*np.tan((mu+np.sin(mu)*(x/2))/2));
    I1 = -(np.pi*cosMu*(tsr - CD*SG*k)*(a - 1) 
           + (CD*CG*cosMu*k_1s*SD*a*k*np.pi*(2*tsr - CD*SG*k))/(8*sinMu))/(2*np.pi)
    I2 = (np.pi*sinMu**2 + (np.pi*(CD**2*CG**2*SD**2*k**2 
                                   + 3*CD**2*SG**2*k**2 - 8*CD*tsr*SG*k 
                                   + 8*tsr**2))/12)/(2*np.pi)
    return (sigma*(cd+cl_alfa)*(I1) - sigma*cl_alfa*theta*(I2)) - x

 Define function to compute $C_P$

In [ ]:
def find_cp(sigma,cd,cl_alfa,gamma,delta,k,cosMu,sinMu,tsr,theta,ct,mu):
    a = 1-((1+np.sqrt(1-ct-1/16*sinMu**2*ct**2))/(2*(1+1/16*ct*sinMu**2)))
    SG = np.sin(np.deg2rad(gamma))
    CG = np.cos(np.deg2rad(gamma))                
    SD = np.sin(np.deg2rad(delta))  
    CD = np.cos(np.deg2rad(delta))  
    k_1s = -1*(15*np.pi/32*np.tan((mu+np.sin(mu)*(ct/2))/2));
    
    cp = sigma*((np.pi*cosMu**2*tsr*cl_alfa*(a - 1)**2 
                 - (tsr*cd*np.pi*(CD**2*CG**2*SD**2*k**2 + 3*CD**2*SG**2*k**2 - 8*CD*tsr*SG*k + 8*tsr**2))/16 
                 - (np.pi*tsr*sinMu**2*cd)/2 - (2*np.pi*cosMu*tsr**2*cl_alfa*theta)/3 
                 + (np.pi*cosMu**2*k_1s**2*tsr*a**2*cl_alfa)/4 
                 + (2*np.pi*cosMu*tsr**2*a*cl_alfa*theta)/3 + (2*np.pi*CD*cosMu*tsr*SG*cl_alfa*k*theta)/3 
                 + (CD**2*cosMu**2*tsr*cl_alfa*k**2*np.pi*(a - 1)**2*(CG**2*SD**2 + SG**2))/(4*sinMu**2) 
                 - (2*np.pi*CD*cosMu*tsr*SG*a*cl_alfa*k*theta)/3 
                 + (CD**2*cosMu**2*k_1s**2*tsr*a**2*cl_alfa*k**2*np.pi*(3*CG**2*SD**2 + SG**2))/(24*sinMu**2) 
                 - (np.pi*CD*CG*cosMu**2*k_1s*tsr*SD*a*cl_alfa*k)/sinMu 
                 + (np.pi*CD*CG*cosMu**2*k_1s*tsr*SD*a**2*cl_alfa*k)/sinMu 
                 + (np.pi*CD*CG*cosMu*k_1s*tsr**2*SD*a*cl_alfa*k*theta)/(5*sinMu) 
                 - (np.pi*CD**2*CG*cosMu*k_1s*tsr*SD*SG*a*cl_alfa*k**2*theta)/(10*sinMu))/(2*np.pi))
    
    return cp

 Set model variables and constants

In [ ]:
delta = -5
sigma = 0.0416
cd              = 0.0051                         # drag coefficient      [-]
c_l_alpha       = 4.7662                         # lift slope            [1/rad]
beta            = -3.336                         # blade twist angle     [deg]

In [ ]:
lut_file = "Cp-Lambda curves 335MW_2/LUT_IEA3MW.npz"
LUT         = np.load(lut_file)
cp_i = LUT['cp_lut']
ct_i = LUT['ct_lut']
pitch_i = LUT['pitch_lut']
tsr_i = LUT['tsr_lut']
interp_lut_cp = RegularGridInterpolator(
    (tsr_i,pitch_i),
    cp_i,
    bounds_error=False,
    fill_value=None
)

In [ ]:
interp_lut_ct = RegularGridInterpolator(
    (tsr_i,pitch_i),
    ct_i,
    bounds_error=False,
    fill_value=None
)

In [ ]:
add_idx = np.array([0,0,1,1])

 Load LES data

In [ ]:
u_inf = np.array([9.5,7,9.5,9.5])      
rho   = 1.17                      
R     = 65   
goal_etaP   = np.zeros((28*4))
goal_ct     = np.zeros((28*4))
tsr_in      = np.zeros((28*4))
pitch_in    = np.zeros((28*4))
gamma_in    = np.zeros((28*4))
shear_in    = np.zeros((28*4))
goal_eta_T  = np.zeros((28*4))
s1 = loadmat('num_data/tsr8/tsr_8_rigid.mat')
s2 = loadmat('num_data/tsr9_5/tsr_9_5_rigid.mat')
s3 = loadmat('num_data/shear01/shear_01_rigid.mat')
s4 = loadmat('num_data/shear03/shear_03_rigid.mat')
goal_etaP[0:28]      = np.squeeze(s1['dati_pow'])
goal_etaP[28*1:28*2] = np.squeeze(s2['dati_pow'])
goal_etaP[28*2:28*3] = np.squeeze(s3['dati_pow'])
goal_etaP[28*3:28*4] = np.squeeze(s4['dati_pow'])
goal_ct[0:28]      = np.squeeze(s1['dati_ct'])/(0.5*rho*np.pi*R**2*u_inf[0]**2)
goal_ct[28*1:28*2] = np.squeeze(s2['dati_ct'])/(0.5*rho*np.pi*R**2*u_inf[1]**2)
goal_ct[28*2:28*3] = np.squeeze(s3['dati_ct'])/(0.5*rho*np.pi*R**2*u_inf[2]**2)
goal_ct[28*3:28*4] = np.squeeze(s4['dati_ct'])/(0.5*rho*np.pi*R**2*u_inf[3]**2)
tsr_in[0:28]      = np.squeeze(s1['dati_tsr'])
tsr_in[28*1:28*2] = np.squeeze(s2['dati_tsr'])
tsr_in[28*2:28*3] = np.squeeze(s3['dati_tsr'])
tsr_in[28*3:28*4] = np.squeeze(s4['dati_tsr'])
pitch_in[0:28]      = np.deg2rad(np.squeeze(s1['dati_theta']))
pitch_in[28*1:28*2] = np.deg2rad(np.squeeze(s2['dati_theta']))
pitch_in[28*2:28*3] = np.deg2rad(np.squeeze(s3['dati_theta']))
pitch_in[28*3:28*4] = np.deg2rad(np.squeeze(s4['dati_theta']))
gamma_in[0:28]      = np.squeeze(s1['dati_gamma'])
gamma_in[28*1:28*2] = np.squeeze(s2['dati_gamma'])
gamma_in[28*2:28*3] = np.squeeze(s3['dati_gamma'])
gamma_in[28*3:28*4] = np.squeeze(s4['dati_gamma'])
shear_in[0:28]      = np.squeeze(s1['dati_k'])
shear_in[28*1:28*2] = np.squeeze(s2['dati_k'])
shear_in[28*2:28*3] = np.squeeze(s3['dati_k'])
shear_in[28*3:28*4] = np.squeeze(s4['dati_k'])
for i in np.arange(len(goal_ct)):
    idx = np.where(( (gamma_in == 0) & (tsr_in == tsr_in[i]) & (np.round(pitch_in,4) == np.round(pitch_in[i],4)) & (shear_in == shear_in[i])))
    dummy = goal_ct[i]/goal_ct[idx[0]]
    goal_eta_T[i] = dummy[0]
    
## Compute $\eta_T$ and $\eta_T$ with the model and compare with LES data
gamma_array    = np.linspace(-30,30,7)
gamma_arrayF   = np.linspace(-30,30,61)
idx0           = np.where(gamma_arrayF == 0)
idx00           = np.where(gamma_array == 0)

In [ ]:
styles   = np.array(["-","--",":","-."])
marks    = np.array(["*","^","o","s"])
legend_a = np.array([r'$\theta_p=1.4^\circ$',r'$\theta_p=4.9^\circ$',r'$\theta_p=6.7^\circ$',r'$\theta_p=8.1^\circ$'])

In [ ]:
paxxo = np.array([1.4,4.9,6.7,8.1])
salva_eta_modello = np.zeros((4,4,len(gamma_arrayF)))

In [ ]:
plt.figure(10,figsize=(10,4.75))
plt.figure(11,figsize=(10,4.75))
plt.figure(20,figsize=(10,4.75))
plt.figure(21,figsize=(10,4.75))

In [ ]:
tsr_array = np.zeros((4,4))
ct0_array = np.zeros((4,4))

In [ ]:
conto_tutto = 0
for i in np.arange(4):
    p_counter = 0
    for p in  np.arange(4):
        eta_p_les = np.zeros(len(gamma_array))
        eta_t_les = np.zeros(len(gamma_array))
        ct_les = np.zeros(len(gamma_array)) 
        c = 0
        for j in gamma_array:
            tsr          = tsr_in    [len(gamma_array)*4*i+len(gamma_array)*p+c]
            gamma        = gamma_in  [len(gamma_array)*4*i+len(gamma_array)*p+c]
            eta_p_les[c] = goal_etaP [len(gamma_array)*4*i+len(gamma_array)*p+c]
            eta_t_les[c] = goal_eta_T[len(gamma_array)*4*i+len(gamma_array)*p+c]
            ct_les[c]    = goal_ct   [len(gamma_array)*4*i+len(gamma_array)*p+c]
            theta        = pitch_in  [len(gamma_array)*4*i+len(gamma_array)*p+c]
            shear        = shear_in  [len(gamma_array)*4*i+len(gamma_array)*p+c]            
            c += 1
        cp = np.zeros(len(gamma_arrayF))
        ct = np.zeros(len(gamma_arrayF))
        aaa  = np.zeros(len(gamma_arrayF))
        c = 0
        for jj in gamma_arrayF:
            gamma = gamma_arrayF[c]
            mu    = np.arccos(np.cos(np.deg2rad(gamma))*np.cos(np.deg2rad(delta)))
            data  = (sigma,cd,c_l_alpha,gamma,delta,shear,np.cos(mu),np.sin(mu),tsr,theta+np.deg2rad(beta),mu)
            x0    = 0.6
            ct[c] = fsolve(find_ct, x0,args=data)[0]
            aaa[c]  = find_a(ct[c],mu)
            cp[c] = find_cp(sigma,cd,c_l_alpha,gamma,delta,shear,np.cos(mu),np.sin(mu),tsr,theta+np.deg2rad(beta),ct[c],mu)
            c += 1     
        if tsr > 9.5:
            tsr = 9.5

        # Heck et al.
        eta_p_Heck = np.zeros(len(gamma_array))
        eta_t_Heck = np.zeros(len(gamma_array))
        c = 0
        for jj in gamma_array:
            gamma = gamma_array[c]
            mu    = np.arccos(np.cos(np.deg2rad(gamma))*np.cos(np.deg2rad(delta)))
            gamma = np.rad2deg(mu)
            eta_p_Heck[c] = Heck_w_C_T(ct_les[c],gamma)[0]
            eta_t_Heck[c] = Heck_w_C_T(ct_les[c],gamma)[1]
            c += 1
        eta_p_Heck = eta_p_Heck/eta_p_Heck[idx00]
        eta_t_Heck = eta_t_Heck/eta_t_Heck[idx00]
        #
        tsr_array[i,p] = tsr
        ct0_array[i,p] = ct_les[idx00]
        eta_t_mod = ct/ct[idx0]        
        eta_p_mod = cp/cp[idx0]
        #
        plt.figure(10+add_idx[i])
        if (i == 2) & (p == 0):
            conto_tutto = 0
        plt.subplot(2,4,conto_tutto+1)
        plt.plot(gamma_arrayF,eta_p_mod,color='C0',linewidth=1.75,label='Present model',zorder=160)
        if add_idx[i] == 0:
            plt.plot(gamma_array[:],eta_p_Heck[:],'--',color='C1',linewidth=1.75,label='Heck et al. (2023)',zorder=140,clip_on=True)
        else:
            plt.plot(gamma_array[:],eta_p_Heck[:],'--',color='C1',linewidth=1.75,label='Heck et al. (2023)',zorder=50,clip_on=True)
        if i < 2:
            plt.scatter(gamma_array[3::],eta_p_les[3::],35,color='k',marker=marks[2],zorder=200,clip_on=False,label='LES')
        else:
            plt.scatter(gamma_array[0::],eta_p_les[0::],35,color='k',marker=marks[2],zorder=200,clip_on=False,label='LES')
        if i < 2:
            plt.xlim([0,30])
            if i == 1:
                plt.xticks(np.linspace(0,30,4))
                plt.xlabel('$\gamma$ [$^\circ$]')
            else:
                plt.xticks(np.linspace(0,30,4),color='w',fontsize=2)
        else:
            plt.xlim([-30,30])
            if i == 3:
                plt.xticks(np.linspace(-30,30,5))
                plt.xlabel('$\gamma$ [$^\circ$]')
            else:
                plt.xticks(np.linspace(-30,30,5),color='w',fontsize=2)        
        plt.ylim([0.6,1.01])
        if p == 0:
            plt.yticks(np.linspace(0.6,1,5))
        else:
            plt.yticks(np.linspace(0.6,1,5),color='w',fontsize=2)                 
        plt.grid()
        if (p==0):
            plt.ylabel('$\eta_P$ [-]')
        plt.tight_layout()
        #        
        plt.figure(20+add_idx[i])
        if (i == 2) & (p == 0):
            conto_tutto = 0
        plt.subplot(2,4,conto_tutto+1)
        plt.plot(gamma_arrayF,eta_t_mod,color='C0',linewidth=1.75,label='Present model',zorder=160)
        if i < 2:
            plt.scatter(gamma_array[3::],eta_t_les[3::],35,color='k',marker=marks[2],zorder=200,clip_on=False,label='LES')
        else:
            plt.scatter(gamma_array[0::],eta_t_les[0::],35,color='k',marker=marks[2],zorder=200,clip_on=False,label='LES') 
        if i < 2:
            plt.xlim([0,30])
            if i == 1:
                plt.xticks(np.linspace(0,30,4))
                plt.xlabel('$\gamma$ [$^\circ$]')
            else:
                plt.xticks(np.linspace(0,30,4),color='w',fontsize=2)
        else:
            plt.xlim([-30,30])
            if i == 3:
                plt.xticks(np.linspace(-30,30,5))
                plt.xlabel('$\gamma$ [$^\circ$]')
            else:
                plt.xticks(np.linspace(-30,30,5),color='w',fontsize=2)        
        plt.ylim([0.7,1.01])
        if p == 0:
            plt.yticks(np.linspace(0.7,1,4))
        else:
            plt.yticks(np.linspace(0.7,1,4),color='w',fontsize=2)                 
        plt.grid()
        if (p==0):
            plt.ylabel('$\eta_T$ [-]')
        conto_tutto += 1
    p_counter += 1
#%%
u = np.array([8,9])
d = np.array([11,12])
for i in np.arange(2):
    plt.figure(10+i)
    plt.subplots_adjust(left=0.08, bottom=0.15, right=0.95, top=0.9, wspace=0.2, hspace=0.1)
    plt.subplot(2,4,1)
    plt.legend(ncol=5,frameon=False,handlelength=1.5,columnspacing=0.6,bbox_to_anchor=(3.8,1.3))
    if i == 0:
        plt.text(140,0.75,'$\lambda=8$',rotation = 90)
        plt.text(140,0.275,'$\lambda=9.5$',rotation = 90)
    else:
        plt.text(250,0.715,'$k=0.06$',rotation = 90)
        plt.text(250,0.265,'$k=0.19$',rotation = 90)
    plt.figure(20+i)
    plt.subplots_adjust(left=0.08, bottom=0.15, right=0.95, top=0.9, wspace=0.2, hspace=0.1)
    plt.subplot(2,4,1)
    plt.legend(ncol=5,frameon=False,handlelength=1.5,columnspacing=0.6,bbox_to_anchor=(2.9,1.3))
    if i == 0:
        plt.text(140,0.815,'$\lambda=8$',rotation = 90)
        plt.text(140,0.455,'$\lambda=9.5$',rotation = 90)
    else:
        plt.text(250,0.815,'$k=0.06$',rotation = 90)
        plt.text(250,0.455,'$k=0.19$',rotation = 90)